Here we'll explore some basic examples of obtaining a numerical solutions to an ODE.  We'll use the scipy.integrate.solve_ivp to do this.  Let's look first at an example of exponential decay, e.g. $\frac{dy}{dt} = -y$.  To start, we'll need to write a function representing the right-side of this *differential equation*.

In [ ]:
from scipy.integrate import solve_ivp

def exponential_decay(t,y,alpha=1):
    return -1*alpha*y

time_interval = (0,10)
initial_conditions = [1]
sol = solve_ivp(exponential_decay,time_interval,initial_conditions)
print(sol)

Let's plot the values return by the numerical solver and compare against the exact solution, $y(t) = e^{-\alpha t}$.

In [ ]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

def exact(t,alpha=1,y0=1):
    return y0*np.exp(-alpha*t)

times_from_solver = sol.t
y_from_solver = sol.y[0]

times = np.linspace(0,10,101)
yvals = exact(times)

fig,ax = plt.subplots()

ax.plot(times_from_solver,y_from_solver,'o')
ax.plot(times,yvals,'r-')
ax.set_xlabel('time')
ax.set_ylabel('fraction remaining')
ax.set_title('Solution to $\dot{y} = -y$')
plt.show()

Now let's consider the case of a simple harmonic oscillator, i.e. $V(x) = \frac{1}{2}kx^2$.  This could be, for example, a spring with force constant $k$.  The motion of a test mass $m$ attached to its end is described by $\frac{d^2 x}{dt^2} = -\omega^2 x$, where $\omega^2 = \frac{k}{m}$.

In [ ]:
def sho(t,x,omega2=1):
    return x[1], -1*omega2*x[0]

time_interval = (0,10)
initial_conditions = [1,0]
sol = solve_ivp(sho,time_interval,initial_conditions)
print(sol)

In [ ]:
times_from_solver = sol.t
x_from_solver = sol.y[0]
v_from_solver = sol.y[1]

def sho_exact_x(t,omega2=1):
    return np.cos(np.sqrt(omega2)*t)

def sho_exact_v(t,omega2=1):
    return -np.sqrt(omega2)*np.sin(np.sqrt(omega2)*t)

times = np.linspace(0,10,101)
xvals = sho_exact_x(times)
vvals = sho_exact_v(times)

fig,(ax0,ax1) = plt.subplots(2,1,sharex=True)

ax0.plot(times_from_solver,x_from_solver,'o')
ax0.plot(times,xvals,'r-')
ax0.set_xlabel('time')
ax0.set_ylabel('x(t)')
ax0.set_title('Solutions to $\ddot{x} = -x$')

ax1.plot(times_from_solver,v_from_solver,'o')
ax1.plot(times,vvals,'r-')
ax1.set_xlabel('time')
ax1.set_ylabel('v(t)')

plt.show()

Now let's look at an anharmonic oscillator, i.e. a potential of the form $V(x;p,\alpha,\beta) = \frac{\alpha}{2} kx^2 + \frac{\beta}{p}\kappa x^p, p \neq 2$, where $\alpha,\beta$ are dimensionless constants of order unity.  Note that below $\omega_1 = \sqrt{\frac{k}{m}}, \omega_2 = \sqrt{\frac{\kappa}{m}}$.

In [ ]:
def aho(t,x,p=4,omega1=1,omega2=0.5,alpha=1,beta=-1):
    return x[1], -alpha*omega1*omega1*x[0] + beta*omega2*omega2*x[0]**(p-1)

time_interval = (0,10)
initial_conditions = [2,0]
sol = solve_ivp(aho,time_interval,initial_conditions)
print(sol)

In [ ]:
times_from_solver = sol.t
x_from_solver = sol.y[0]
v_from_solver = sol.y[1]

fig,(ax0,ax1) = plt.subplots(2,1,sharex=True)

ax0.plot(times_from_solver,x_from_solver,'o')
ax0.set_xlabel('time')
ax0.set_ylabel('x(t)')
ax0.set_title('Solutions to $\ddot{x} = -x - x^2$')

ax1.plot(times_from_solver,v_from_solver,'o')
ax1.set_xlabel('time')
ax1.set_ylabel('v(t)')

plt.show()

Now we'll look at a damped oscillator, e.g. $F(t,x,\dot{x}) = -kx - c\dot{x}$.

In [ ]:
def damped_oscillator(t,x,m=1.,k=1.,c=0.1):
    return x[1], -1*(k/m)*x[0] - 2.*c*np.sqrt(k/m)*x[1]/2./np.sqrt(m*k)

time_interval = (0,100)
initial_conditions = [1,0]
t_eval = np.linspace(0,100,1001)

sol = solve_ivp(damped_oscillator,time_interval,initial_conditions,t_eval=t_eval)
print(sol.success)

In [ ]:
def damped_exact_x(t,m=1.,k=1.,c=0.1):
    spring_constant = k/m
    damping_constant = c/2./m
    omega = np.sqrt(spring_constant**2 - damping_constant**2)
    x0 = initial_conditions[0]
    v0 = (initial_conditions[1]+damping_constant*x0)/omega
    return (x0*np.cos(omega*t) + v0*np.sin(omega*t))*np.exp(-damping_constant*t)

def damped_exact_v(t,m=1.,k=1.,c=0.1):
    spring_constant = k/m
    damping_constant = c/2./m
    omega = np.sqrt(spring_constant**2 - damping_constant**2)
    x0 = initial_conditions[0]
    v0 = (initial_conditions[1]+damping_constant*x0)/omega
    return (-omega*x0*np.sin(omega*t) + omega*v0*np.cos(omega*t))*np.exp(-damping_constant*t) - damping_constant*(x0*np.cos(omega*t)+v0*np.sin(omega*t))*np.exp(-damping_constant*t)

times = np.linspace(0,100,1001)
xvals = damped_exact_x(times)
vvals = damped_exact_v(times)

times_from_solver = sol.t
x_from_solver = sol.y[0]
v_from_solver = sol.y[1]

fig,(ax0,ax1) = plt.subplots(2,1,sharex=True)

ax0.plot(times_from_solver,x_from_solver,'o')
ax0.plot(times,xvals,'r-')
ax0.set_xlabel('time')
ax0.set_ylabel('x(t)')
ax0.set_title('Solutions to $\ddot{x} = -x - 2\dot{x}$')

ax1.plot(times_from_solver,v_from_solver,'o')
ax1.plot(times,vvals,'r-')
ax1.set_xlabel('time')
ax1.set_ylabel('v(t)')

plt.show()

Now let's look at a forced oscillator with forcing function $F(t,x,\dot{x}) = F_0\cos(\omega_f t)$, where $F_0$ is the (constant) amplitude of the driver and $\omega_f$ is its frequency.

In [ ]:
def driven_oscillator(t,x,m=1.,k=1.,f0=0.1,omegaf=1):
    return x[1],-1*(k/m)*x[0] + f0*np.cos(omegaf*t)

time_interval = (0,20)
initial_conditions = [1,0]
t_eval = np.linspace(0,20,201)

sol = solve_ivp(driven_oscillator,time_interval,initial_conditions,t_eval=t_eval)
print(sol.success)

In [ ]:
#times = np.linspace(0,100,1001)
#xvals = damped_exact_x(times)
#vvals = damped_exact_v(times)

times_from_solver = sol.t
x_from_solver = sol.y[0]
v_from_solver = sol.y[1]

fig,(ax0,ax1) = plt.subplots(2,1,sharex=True)

ax0.plot(times_from_solver,x_from_solver,'o')
#ax0.plot(times,xvals,'r-')
ax0.set_xlabel('time')
ax0.set_ylabel('x(t)')
ax0.set_title('Solutions to $\ddot{x} = -x - 2\dot{x}$')

ax1.plot(times_from_solver,v_from_solver,'o')
#ax1.plot(times,vvals,'r-')
ax1.set_xlabel('time')
ax1.set_ylabel('v(t)')

plt.show()

In [ ]:
time_interval = (0,100)
initial_conditions = [1,0]
t_eval = np.linspace(0,100,1001)

sol = solve_ivp(driven_oscillator,time_interval,initial_conditions,t_eval=t_eval,args=(1.,1.,100,1.))
print(sol.success)

In [ ]:
times_from_solver = sol.t
x_from_solver = sol.y[0]
v_from_solver = sol.y[1]

fig,(ax0,ax1) = plt.subplots(2,1,sharex=True)

ax0.plot(times_from_solver,x_from_solver,'o')
ax0.set_xlabel('time')
ax0.set_ylabel('x(t)')
ax0.set_title('Solutions to $\ddot{x} = -x - 2\dot{x}$')

ax1.plot(times_from_solver,v_from_solver,'o')
ax1.set_xlabel('time')
ax1.set_ylabel('v(t)')

plt.show()

In [ ]:
time_interval = (0,100)
initial_conditions = [1,0]
t_eval = np.linspace(0,100,1001)

sol = solve_ivp(driven_oscillator,time_interval,initial_conditions,t_eval=t_eval,args=(1.,1.,100,1./np.pi))
print(sol.success)

In [ ]:
times_from_solver = sol.t
x_from_solver = sol.y[0]
v_from_solver = sol.y[1]

fig,(ax0,ax1) = plt.subplots(2,1,sharex=True)

ax0.plot(times_from_solver,x_from_solver,'o')
ax0.set_xlabel('time')
ax0.set_ylabel('x(t)')
ax0.set_title('Solutions to $\ddot{x} = -x - 2\dot{x}$')

ax1.plot(times_from_solver,v_from_solver,'o')
ax1.set_xlabel('time')
ax1.set_ylabel('v(t)')

plt.show()